In [1]:
import random
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Linear Regression from scratch
$f(x) = w*x + b$

In [2]:
def forward(x, w, b):
  return w * x + b

def loss(y, y_pred):
  return ((y - y_pred) ** 2).mean()  # Mean Squared Error

In [3]:
# Training
def train(X, y, w, b, n_epochs: int = 100, learning_rate: float = 0.01, print_loss: bool = False):
  for epoch in range(n_epochs):
    # predict = forward pass
    y_pred = forward(X, w, b)

    # loss
    l = loss(y, y_pred)

    # calculate gradients = backward pass
    l.backward()

    # update parameters (weights)
    # w.data = w.data - learning rate * w.grad
    with torch.no_grad():
      w -= learning_rate * w.grad
      b -= learning_rate * b.grad

    # zero the gradients after updating
    w.grad.zero_()
    b.grad.zero_()

    if print_loss and (epoch + 1) % 10 == 0:
      print(f'epoch {epoch + 1}:\tw = {w.item()},\tb = {b.item()},\tloss = {l.item()}')

  return w, b

def predict(X, w, b):
  return forward(X, w, b).item()

In [4]:
# f(x) = 2 * x
def f(x):
  return 2 * x

abscissae: list[float] = list(range(1, 6))
ordinates: list[float] = [f(x) for x in abscissae]

abscissae, ordinates

([1, 2, 3, 4, 5], [2, 4, 6, 8, 10])

In [5]:
X = torch.tensor(abscissae, dtype=torch.float32)
y = torch.tensor(ordinates, dtype=torch.float32)

# Parameters
w = torch.tensor(random.random() * 0.01, dtype=torch.float32, requires_grad=True)
b = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

w, b

(tensor(0.0083, requires_grad=True), tensor(0., requires_grad=True))

In [6]:
X_test = 4.5
print(f'Prediction before training: f({X_test}) = {predict(X_test, w, b)}')
w, b = train(X, y, w, b, n_epochs=1000, learning_rate=0.001, print_loss=False)
print(f'Prediction before training: f({X_test}) = {predict(X_test, w, b)}')
print(f'slope = {w.item()}, intercept = {b.item()}')

Prediction before training: f(4.5) = 0.037500232458114624
Prediction before training: f(4.5) = 8.909967422485352
slope = 1.898803472518921, intercept = 0.36535120010375977


# Linear Regression using `torch.nn`

In [7]:
import torch.nn as nn

In [8]:
# f(x) = 2 * x
def f(x):
  return 2 * x

abscissae: list[float] = [[i] for i in range(1, 6)]
ordinates: list[float] = [[f(x[0])] for x in abscissae]

abscissae, ordinates

([[1], [2], [3], [4], [5]], [[2], [4], [6], [8], [10]])

In [9]:
X = torch.tensor(abscissae, dtype=torch.float32)
y = torch.tensor(ordinates, dtype=torch.float32)

n_samples, n_features = X.shape
print(f'n_samples = {n_samples}, n_features = {n_features}')

X_test = torch.tensor([4.5], dtype=torch.float32)

n_samples = 5, n_features = 1


In [10]:
# Designing the model
class LinearRegression(nn.Module):
  def __init__(self, input_dim, output_dim) -> None:
    super(LinearRegression, self).__init__()

    # Define different layers
    self.lin = nn.Linear(input_dim, output_dim)

  def forward(self, x):
    return self.lin(x)


input_size = output_size = n_features

model = LinearRegression(input_size, output_size)

print(f'Prediction before training: f({X_test.item()}) = {model(X_test).item()}')

# Define loss and optimizer
learning_rate = 0.01
n_epochs = 1000

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(n_epochs):
  # predict = forward pass with our model
  y_predicted = model(X)

  # loss
  l = loss(y, y_predicted)

  # calculate gradients = backward pass
  l.backward()

  # update parameters
  optimizer.step()

  # zero the gradients after updating
  optimizer.zero_grad()

  if (epoch + 1) % (n_epochs // 10) == 0:
    w, b = model.parameters()  # unpack parameters
    print(f'epoch: {epoch + 1}:\tw = {w[0][0].item()}\tloss = {l.item()}')

print(f'Prediction after training: f({X_test.item()}) = {model(X_test).item()}')

Prediction before training: f(4.5) = 0.09508007764816284
epoch: 100:	w = 1.979898452758789	loss = 0.0009651852888055146
epoch: 200:	w = 1.9856733083724976	loss = 0.0004902765504084527
epoch: 300:	w = 1.9897890090942383	loss = 0.0002490445622242987
epoch: 400:	w = 1.992722511291504	loss = 0.00012650371354538947
epoch: 500:	w = 1.994813323020935	loss = 6.426281470339745e-05
epoch: 600:	w = 1.9963032007217407	loss = 3.2642677979310974e-05
epoch: 700:	w = 1.9973652362823486	loss = 1.6582906027906574e-05
epoch: 800:	w = 1.9981220960617065	loss = 8.42366716824472e-06
epoch: 900:	w = 1.9986615180969238	loss = 4.2785950427060015e-06
epoch: 1000:	w = 1.999045968055725	loss = 2.173748498535133e-06
Prediction after training: f(4.5) = 8.999150276184082
